## Lede Algorithms 2018 Week 2 class 2 - Logistic Regression

This notebook builds up to logistic regression, that is, fitting a model that predicts the output where there are only two choices. But before that, we're going to do many comparisons and visualizations to talk about the underlying ideas that regression is getting at -- especially the idea of "controlling for" another factor. 

Throughout we'll be using data on citations for speeding from Massechusets from 2001. This data was originally analyzed by the Boston Globe for their series [Speed Trap: Who gets a ticket, who gets a break?](http://archive.boston.com/globe/metro/packages/tickets/). You can read about their analysis in this [methodology document](http://archive.boston.com/globe/metro/packages/tickets/study.pdf), and get the original data [here](http://archive.boston.com/globe/metro/packages/tickets/warnings.zip). I've added a header row the CSV file, but otherwise `tickets-warnings.csv` in this directory is identical.

Why such old data? Well, for one thing it's a classic example of multi-variable regression in journalism. But also, for a brief period of April and May 2001, the state government entered both tickets and warnings from the paper citations. In Massechusets at that time, both were written on the same form, with only a checkbox indicating which it was. A ticket means a fine and a raises your insurance premium. A warning means nothing. Having data on both means we can ask who gets a ticket vs. a warning for the same circumstances.

We have a ton of data about each stop, including the speed, age, race, gender, and home zip code of the driver. The journalistic question is, are some groups more likely to get tickets, and not warnings, than others?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
tw_raw = pd.read_csv('tickets-warnings.csv')
tw_raw

For a complete description of all 57 fields, see the [methodology](http://archive.boston.com/globe/metro/packages/tickets/study.pdf). For our purposes, we are only intersted in the citations for speeding, and most interested in the fields `TYPE` (ticket or warning), `MINORITY`, `SEX`, `AGE`, and `MPHOVER`.

In [ ]:
tw = tw_raw[tw_raw['DESCRIPT']=='SPEEDING'].reset_index(drop=True)
tw = tw[['DATE', 'TYPE', 'AGENCY3', 'SEX','MINORITY','AGE','MPH','MPHOVER','INTOWN']]
tw.head()

Let's start with the total number of minority vs. white drivers

In [ ]:
tw.groupby('MINORITY').count()

What fraction of citations were given to minorities? 

In [ ]:
12836/(12836+79275)

Is 13% a lot or a little? We could compare to the minority population of the towns where these stops happened, but what about people driving through? And there's no guarantee drivers in both groups have the same distribution of speeds. 

This is often called the "denominator problem." To deal with this issue, we're going to compare the probability of getting a warning, as opposed to a ticket, between the groups.

First, let's clean up the data a little but dropping the (relatively small) number of records where the minority status is unknown.

In [ ]:
# drop the unknown race entries
tw = tw[tw['MINORITY'] != 'U'].reset_index(drop=True)  # renumber the rows, and don't create a new column

In [ ]:
sz = tw.groupby(['MINORITY','TYPE']).size()
sz

In [ ]:
pd.crosstab(tw.MINORITY,tw.TYPE)

In [ ]:
pd.crosstab(tw.MINORITY,tw.TYPE).plot(kind='bar')

In [ ]:
# percent of minority citations which were tickets
sz['M']['T'] / sum(sz['M'])

In [ ]:
# percent of non-minority citations which were tickets
sz['W']['T'] / sum(sz['W'])

We can make a nice little chart of this...

In [ ]:
pd.DataFrame([sz['M']['T'] / sum(sz['M']), sz['W']['T'] / sum(sz['W'])], index=['Minority','White']).plot(kind='bar')

But how do we know that the drivers in these two groups are really the same? We imagine that there are more tickets given to faster drivers (`MPHOVER` is how much over the speed limit the driver was). Legend also has it that traffic cops are harsher to drivers from out of town (`INTOWN` records where the driver was from.)

To get a sense of this, let's compare histograms of the two groups.

In [ ]:
tw['MPHOVER'].hist()

We've used `MPHOVER` because speed over the limit is probably more interesting than how fast in absolute terms. But checking raw `MPH`, we see a problem with the data: some of the entries don't record the speed! Fortunately, not very many. 

In [ ]:
tw['MPH'].hist()

To see if the distributions are different between white and minorities, let's try plotting them on the same chart.

In [ ]:
w_over = tw[tw['MINORITY']=='W']['MPHOVER'].dropna()  # need dropna because some rows don't record speed
m_over = tw[tw['MINORITY']=='M']['MPHOVER'].dropna()

bins = np.linspace(0, 100, 20)
plt.hist(w_over,bins,alpha=0.5)
plt.hist(m_over,bins,alpha=0.5)
plt.show()

Still very hard to see, because the scaling is so different. Let's normalize the histograms to display percentages instead of counts.

In [ ]:
plt.hist(w_over,bins,alpha=0.5,density=True)
plt.hist(m_over,bins,alpha=0.5,density=True)


Hmm, they look pretty close. The minorities plot has a slightly fatter tail, so that's some evidence that they're driving faster, but it doesn't seem to be a huge difference. Let's take a look at the averages, just to get another sense.

In [ ]:
w_over.mean()

In [ ]:
m_over.mean()

So, about half a mile an hour on average difference. Still, this could make the difference if, for example, it pushes a lot of drivers greater than some threshold like 20 mph over the limit. We're going to need to untangle the effects here.

In fact there could be all sorts of things going on here. Age and sex could be factors. In town or out of town drivers could matter. The `AGENCY` column records whether the stop was made by Massechusets state police, Boston police, or all other police departments, and that could matter both in terms of how the different agencies operate and in terms of who is driving within their jurisdiction. 

For our purposes, bias means that minorities get a ticket more often than white people *when everything else is equal.* Is everything else equal? Let's plot a pile of histograms to take a look.

In [ ]:
def comparehist(colname):
    w = tw[tw['MINORITY']=='W'][colname].dropna()
    m = tw[tw['MINORITY']=='M'][colname].dropna()
    
    if tw[colname].dtype == np.float64:
        # it's numeric, plot a histogram
        bins = np.linspace(0, 100, 20)
        plt.hist(w,bins,alpha=0.5,density=True)
        plt.hist(m,bins,alpha=0.5,density=True)
    else:
        # if not numerican, assume it's categorical and plot a bar chart of value counts normalized to percentages
        wc = w.value_counts()
        wc /= sum(wc)
        plt.bar(wc.index, wc, alpha=0.5)
        mc = m.value_counts()
        mc /= sum(mc)
        plt.bar(mc.index, mc, alpha=0.5)
        
    plt.title(colname)
    plt.show()


In [ ]:
comparehist('AGE')
comparehist('SEX')
comparehist('MPH')
comparehist('MPHOVER')
comparehist('AGENCY3')
comparehist('INTOWN')


What can we say here? The minority drivers in this sample have more people in their 20s, skew male, are going slightly faster when they're pulled over, are less likely to be stopped by local police and more likely to be stopped by Boston police, and tend to be out of town more often. Any or all of these effects could account for the difference in ticketing rates. There could even be interactions here: suppose Boston police are harder on people who drive more than 10 mph over the limit. These minority drivers are both more commonly in Boston and going faster on average.

### Enter logistic regression
We're going to use regression to try to analyze the relationships between the variables and whether someone gets a ticket. But everything is going to be a little bit different than last class because the variable we are going to try to predict is binary: it's a yes/no, did they get a ticket or not, instead of a numeric value.

In [ ]:
from sklearn.linear_model import LinearRegression,LogisticRegression



To set up the regression, let's look at a graph of MPH over the limit on the x axis, vs. whether the driver got a ticket on the y axis, represented as 0 or 1. 

In [ ]:
# add a column that is 1 if they got a ticket (not a warning)
tw['BUSTED'] = tw['TYPE'].replace({'W':0, 'T':1})

# Remove rows where MPH is unknown
tw = tw[~tw['MPHOVER'].isnull()].reset_index()

In [ ]:
# sort by MPHOVER (makes the plotting below simpler)
tw = tw.sort_values(by='MPHOVER')

In [ ]:
x = tw[['MPHOVER']].values
y = tw[['BUSTED']].values
tw.plot(kind='scatter',x='MPHOVER',y='BUSTED')

This has a general "upward slope." Let's use linear regression to try to fit a trend line to it.

In [ ]:
lm = LinearRegression()
lm.fit(x,y)
tw.plot(kind='scatter',x='MPHOVER',y='BUSTED')
plt.plot(x,lm.predict(x),'-',color='m')

This is a little strange, because we're trying to predict binary outcome with a straight line. If you like, you can imagine that whenever this line is 0.5 or greater, we guess "got a ticket." But really we should use a model that always predicts a value between 0 and 1 -- a probability. 

To do this we fit a "logistic" function, not a line. This is a function which is an S-shape, and turns any number into a number from 0 to 1

In [ ]:
def logistic(x):
    return 1 / (1 + np.exp(-x))

linear_range = np.linspace(-10,10)
plt.plot(linear_range, logistic(linear_range))

The exact center of this function, at x=0, has a probability of 0.5, meaning "I'm indifferent guessing between these two options."

In [ ]:
logistic(0)

Let's apply logistic regression and plot the answer. To interpret the output, we take our normal `y=mx+b` line but run it through the logistic function.

In [ ]:
# Run the logistic regression on our data
lm = LogisticRegression()
lm.fit(x,y)

# plot original data
tw.plot(kind='scatter',x='MPHOVER',y='BUSTED')

# Now plot the prediction of the fitted model
m = lm.coef_[0]
b = lm.intercept_
line_y = m*x+b
prediction = logistic(line_y) 
plt.plot(x, prediction, '-', color='m')

We can do the same thing with the `predict_proba` function (though it returns two columns: one is the probability of  FALSE and the other is the probability of TRUE, so we plot only the second column)

In [ ]:
tw.plot(kind='scatter',x='MPHOVER',y='BUSTED')
plt.plot(x, lm.predict_proba(x)[:,1], '-', color='m')

### Odds ratios

Here's a useful fact: the "slope" coefficients of a logistic regression means something when the indepenent variable is binary: it's the *odds ratio*.

Suppose we have a dinner party where each person eats chicken or fish. But oh no! Some people got sick afterwards. To see if it was related to what they ate, we can calculate the odds ratio.

In [ ]:
# the fish column is 0 if they ate chicken, 1 if fish
dinner = pd.DataFrame([[0,1],[0,0],[0,0],[1,0],[1,1],[1,1]], columns=['fish','sick'])
dinner

The "odds" of getting sick if you ate chicken are just the number of people who ate chicken and got sick, divided by the number of people who ate chicken and did not get sick.

In [ ]:
# useful little function
ct = pd.crosstab(dinner.fish, dinner.sick)
ct

In [ ]:
# odds of getting sick if you ate chicken
chicken_odds = ct[0][1] / ct[0][0]
chicken_odds

In [ ]:
# odds of getting sick if you ate fish
fish_odds = ct[1][1] / ct[1][0]
fish_odds

Then "odds ratio" is, yup, the ratio of these sets of odds. It measures how your odds of getting sick *changed* if you ate the fish. 

In [ ]:
fish_odds / chicken_odds

Now let's do a logistic regression to predict who gets sick

In [ ]:
x=dinner[['fish']].values
y=dinner[['sick']].values
lm = LogisticRegression(C=1e5) # this C thing... we just need some large value here, it's a long story
lm.fit(x,y)

In [ ]:
# To get the odds ratio, we use the following formula:
np.exp(lm.coef_)



### Controlling for other variables

We can feed as many independent variables as we like into linear regression. With two variables, instead of fitting a plane to the data we are fitting a curved surface. 

But here's the cool thing: just like linear regression each coefficient tells us how much that independent variable effects the probability of getting a ticket *if all other variables stay the same.*



In [ ]:
# Code these variables a binary
tw.SEX = tw.SEX.replace({'M':0,'F':1, 'U':0})
tw.MINORITY = tw.MINORITY.replace({'W':0,'M':1})
tw.INTOWN = tw.INTOWN.replace({'N':0,'Y':1, 'U':0})

# remove unknown ages
tw = tw[~tw.AGE.isnull()].reset_index()

In [ ]:
x = tw[['MPHOVER','MINORITY','SEX', 'AGE', 'INTOWN']].values
y = tw[['BUSTED']].values
lm = LogisticRegression() 
lm.fit(x,y)
       
# print out the odds ratios for each variable
np.exp(lm.coef_)